<a href="https://colab.research.google.com/github/drfcozapata/assets/blob/master/Drive_Drive_Mega_Drive_Drive_Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#@markdown <center></br><img src="https://drfcozapata.files.wordpress.com/2020/12/logo-01a.png" height="60" alt="Logo DrFcoZapata"/></center>

# Agregar Google Drive a la Maquina Virtual de Colab

Aqui iniciarás sesión con tu cuenta de Google para poder tener acceso a tus archivos

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Agregar Google Drive /content/drive</h3></center><br>
MODO = "MONTAR" #@param ["MONTAR", "DESMONTAR"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODO == "MONTAR":
  drive.mount('/content/drive', force_remount=True)
elif MODO == "DESMONTAR":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")


# Copiar contenido entre Drive y Unidades ilimitadas
Abre la carpeta del Drive en la columna de la izquierda, selecciona la que deseas copiar; en el menú contextual (click derecho) selecciona "Copiar la ruta" y luego pégala en "RUTA_ORIGEN". Haz lo mismo con el sitio donde deseas copiarla y pega la ruta en "RUTA_DESTINO".



In [ ]:
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h2>Copiar contenido entre Unidades</h2></center><br>
RUTA_ORIGEN = "" #@param {type:"string"}
RUTA_DESTINO = "" #@param {type:"string"}
import shutil

# Copia el archivo desde la ubicación actual a la
# carpeta "Documentos".
shutil.copy(RUTA_ORIGEN, RUTA_DESTINO)

# Descomprimir/Comprimir archivos 7Z, RAR, TAR o ZIP en Google Drive

In [ ]:
#@markdown <center><img src='https://drfcozapata.files.wordpress.com/2020/12/7z.png' height="50" alt="Logo-7z"/>&nbsp;<img src='https://drfcozapata.files.wordpress.com/2020/12/rar.png' height="50" alt="Logo-RAR"/>&nbsp;<img src='https://drfcozapata.files.wordpress.com/2020/12/tar.png' height="50" alt="Logo-TAR"/>&nbsp;<img src='https://drfcozapata.files.wordpress.com/2020/12/zip.png' height="50" alt="Logo-ZIP"/></center>
#@title Descomprimir archivos 

# ============================= FORM ============================= #

MODO = "UNRAR"  # @param ["7Z", "UNRAR", "UNTAR", "UNZIP"]
RUTA_ORIGEN = ""  # @param {type:"string"}
RUTA_DESTINO = ""  # @param {type:"string"}
PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global RUTA_DESTINO
    if PASSWORD:
      passADD = f'-P {PASSWORD}'
    else:
      passADD = ''
    if not RUTA_DESTINO:
      RUTA_DESTINO = "/content/extract"
    os.makedirs(RUTA_DESTINO, exist_ok=True)
    if MODO == "UNRAR":
        runSh(f'unrar x "{RUTA_ORIGEN}" "{RUTA_DESTINO}" '+passADD+' -o+', output=True)
    elif MODO == "UNTAR":
        runSh(f'tar -C "{RUTA_DESTINO}" -xvf "{RUTA_ORIGEN}"', output=True)
    elif MODO == "UNZIP":
        runSh('unzip '+passADD+f' "{RUTA_ORIGEN}" -d "{RUTA_DESTINO}"', output=True)
    else:
        runSh(f'7z x "{RUTA_ORIGEN}" -o{RUTA_DESTINO} '+passADD, output=True)

extractFiles()

# Login en Mega para los procesos que siguen

In [ ]:
#@markdown <center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
# @markdown <center><h3>Ingresa los datos de tu cuenta de Mega </h3></center><br>
# @markdown *Después de ingresar a tu cuenta, se usará tu cuota de transferencia si vas a importar al Drive; te sugerimos que uses una cuenta Pro de Mega. La cuota de transferencia de Google Drive para exportar tiene un limite de 750Gb.*
# @markdown  
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID
USUARIO = ""  # @param {type:"string"}
CONTRASEÑA = ""  # @param {type:"string"}
USERNAME=USUARIO
PASSWORD=CONTRASEÑA
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Por favor, ingresa los datos de tu cuenta de Mega.")


# Importar link de Mega a Drive

In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>


#@markdown <center><h2>Transferir de Mega a Google Drive</h2></center><br>
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link downloa
LINK_DE_MEGA="" #@param {type:"string"}
RUTA_DESTINO="" #@param {type:"string"}
URL = LINK_DE_MEGA
OUTPUT_PATH =RUTA_DESTINO 
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()



#Exportar link de Drive a Mega

In [ ]:
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>

RUTA_ORIGEN = ""  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    cmd = ['mega-put', RUTA_ORIGEN, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")
